# Instance Segmentation with YOLO11
This notebook has two features:
- Training a YOLO model on the dataset
- Predicting with the trained YOLO model

## 0. Initialization
Link to models: https://docs.ultralytics.com/tasks/segment/
- We used **YOLO11m-seg**
- Download it if you want to train the model from scratch

In [ ]:
%pip install ultralytics

In [7]:
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import shutil
import os
import numpy as np

In [1]:
path_to_pre_trained_model = "CONFIGURE_PATH_TO_DOWNLOADED_MODEL" # This is the model downloaded from the link provided above
dataset = "../data/model_training/YOLODataset/dataset.yaml" 
path_to_trained_yolo_model = "../checkpoints/best_yolo_model.pt"
output_folder = "../data/predicted/YOLO/masks"

## 1. Training

### 1.1 Convert Annotations from JSON to YOLO & Split Into Train, Val, and Test Sets

**IMPORTANT!**

You should have **already completed the preprocessing steps** from the preprocess notebook. 

[labelme2yolo](https://pypi.org/project/labelme2yolo/) is used to split the dataset into train, test, and validation sets. The following code will set up labelme2yolo and run it to create training, testing, and validation sets 

In [ ]:
%pip install labelme2yolo

In [ ]:
!labelme2yolo --json_dir "../data/processed/scaled/labels" --val_size 0.15 --test_size 0.15 --output_format polygon --seed 42

In [ ]:
source_folder = "../data/processed/scaled/labels/YOLODataset"
destination_folder = "../data/model_training"

os.makedirs(destination_folder, exist_ok=True)

try:
    shutil.move(source_folder, destination_folder)
    print(f"Moved '{source_folder}' to '{destination_folder}'.")
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure the source folder exists.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


In [ ]:
def rewrite_dataset_yaml(yolo_dataset_path, yaml_file_path):
    abs_path = os.path.abspath(yolo_dataset_path)

    # New content for the YAML file
    yaml_content = f"""
path: {abs_path}
train: images/train
val: images/val
test: images/test

names:
    0: window
    """

    try:
        with open(yaml_file_path, "w") as yaml_file:
            yaml_file.write(yaml_content.strip())
        print(f"Rewritten dataset.yaml successfully at {yaml_file_path}.")
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

yolo_dataset_path = "../data/model_training/YOLODataset"  
yaml_file_path = "../data/model_training/YOLODataset/dataset.yaml"    
rewrite_dataset_yaml(yolo_dataset_path, yaml_file_path)


### 1.2 Train Model
Results will be saved to the **root** of the project **/runs/train**


Prerequisites:
- You have configured a path to the pre-trained model, downloaded from the Ultralytics website
- You have completely finished the preprocessing steps and followed the steps above

In [ ]:
torch.cuda.empty_cache()

model = YOLO(path_to_pre_trained_model) # Path to pre-trained model

model.train(data = dataset, imgsz = 640, device = 0, batch = 8, epochs = 50, workers = 1)

## 2. Predict with Model
Results will be saved to `../data/predicted/YOLO/masks`


To see more inference arguments, please see:
- https://docs.ultralytics.com/modes/predict/#inference-arguments

In [3]:
# So images and graphs can be printed in the notebook
%matplotlib inline

In [12]:
def predict_with_yolo(image_path, output_folder=None, skip_display=True):
    model = YOLO(path_to_trained_yolo_model) # Path to your trained model

    results = model.predict(
        source=image_path,  # Path to the single image you want to predict
        show=False,         # Open image with drawn mask in a new window
        save=False,         # Save image with drawn mask
        conf=0.6,           # Minimum confidence score 
        line_width=1,       # Width of box line
        save_crop=False,    # Save window crops 
        save_txt=False,     # Save bbox and segmentation label
        show_boxes=True,    # Show bbox
        show_labels=True,   # Show labels
        show_conf=True,     # Show confidence score
        classes=[0],        # Which classes to include (we only have 1, hence [0])
        iou=0.6             # Minimum IoU score
    )                       # More can be configured, please see provided link above

    # Process and save predicted mask
    masks = results[0].masks.data  

    
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    combined_mask = np.zeros((height, width), dtype=np.uint8)

    
    for mask in masks:
        mask_np = mask.cpu().numpy()
        mask_np = (mask_np > 0).astype(np.uint8) * 255  
        mask_resized = cv2.resize(mask_np, (width, height), interpolation=cv2.INTER_LINEAR)
        combined_mask = np.maximum(combined_mask, mask_resized)

    base_name = os.path.basename(image_path)  
    file_name_without_extension = os.path.splitext(base_name)[0]  

    if output_folder:
        os.makedirs(output_folder, exist_ok=True)  
        output_path = os.path.join(output_folder, f"{file_name_without_extension}_mask.png")
        cv2.imwrite(output_path, combined_mask)

    if not skip_display:
        # Display the combined mask
        plt.imshow(combined_mask, cmap='gray')
        plt.axis('off')
        plt.show()

    print(f"Predicted mask saved to {output_folder}")

### 2.1 Predict on Single Image
Replace `path_to_single_image_predict` with the path of the single image you want to predict

In [ ]:
path_to_single_image_predict = "configure_path_here"
predict_with_yolo(path_to_single_image_predict, skip_display=False)

### 2.2 Predict on Batch of Images
Replace `path_to_batch_predict` with the path to the folder containing images to predict

In [ ]:
path_to_batch_predict = "configure_path_here"
files = os.listdir(path_to_batch_predict)

image_extensions = ['.jpeg', '.jpg', '.png']
image_files = [f for f in files if any(f.lower().endswith(ext) for ext in image_extensions)]

for image_filename in image_files:
    image_path = os.path.join(path_to_batch_predict, image_filename)
    
    try:
        predict_with_yolo(image_path, output_folder)  
    except Exception as e:
        # In case of error, print the error with the image path and error message
        print(f"Error processing {image_path}: {str(e)}")